### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [28]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - ((l_c ** 2 / l_s).sum(axis=1)
            + (r_c ** 2 / r_s).sum(axis=1)) / (l_s + r_s)[0]
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return - (((l_c / l_s).log2() * l_c).sum(axis=1)
                  + ((r_c / r_s).log2() * r_c).sum(axis=1)) / (l_s + r_s)[0]

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (l_c.max(axis=1) + r_c.max(axis=1)) / (l_s + r_s)[0]
    
    def __get_feature_importance(self, y):
        _, counts = np.unique(y, return_counts=True)
        if self.G_function == self.__gini:
            return y.size * (1 - np.sum((counts / y.size) ** 2))
        elif self.G_function == self.__entropy:
            return - y.size * (np.sum((counts / y.size)
                                  * np.log2(counts / y.size)))
        elif self.G_function == self.misclass:
            return y.size * (1 - np.max(counts / y.size))

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feauture_ids[:int(np.sqrt(n_feature))])
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feauture_ids[:int(np.log2(n_feature))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]
    
    def __find_threshold(self, x, y):
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)

        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)

        if len(r_border_ids) == 0:
            return np.inf, None

        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)

        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        g_value = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(g_value)

        left_el_id = l_sizes[idx][0]
        return g_value[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0
    
    def make_leaf_node(self, y):
        bin_count = np.bincount(y)
        leaf_node = (self.LEAF_TYPE, bin_count.argmax(), bin_count)
        return leaf_node

    def __fit_node(self, x, y, node_id, depth):
        
        if (self.max_depth and depth >= self.max_depth) or (self.min_samples_split > y.shape[0]):
            self.tree[node_id] = self.make_leaf_node(y)
            return
        
        feature_ids = self.get_feature_ids(x.shape[1])
        threshold = np.array([self.__find_threshold(x[:, feature_id], y) for feature_id in feature_ids])
        cutoff_id = np.argmin(threshold[:, 0])
        _, cutoff = threshold[cutoff_id]
        
        if cutoff is None:
            self.tree[node_id] = self.make_leaf_node(y)
            return
        
        xl, xr, yl, yr = self.__div_samples(x, y, feature_ids[cutoff_id], cutoff)
        
        if yl.shape[0] == 0 or yr.shape[0] == 0:
            self.tree[node_id] = self.make_leaf_node(y)
            return
        
        self.feature_importances_[cutoff_id] += self.__get_feature_importance(y) \
            - self.__get_feature_importance(yl) \
            - self.__get_feature_importance(yr)
        
        node = (self.NON_LEAF_TYPE, feature_ids[cutoff_id], cutoff)
        self.tree[node_id] = node
        self.__fit_node(xl, yl, node_id * 2 + 1, depth+1)
        self.__fit_node(xr, yr, node_id * 2 + 2, depth+1) 
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [224]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [9]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [160]:
%time clf.fit(X_train, y_train)

Wall time: 8 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [222]:
%time my_clf.fit(X_train, y_train)

Wall time: 61 ms


## Проверка качества работы на wine

In [226]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9440559440559441

In [230]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8885003885003885

## Подготовка данных Speed Dating Data 

In [12]:
df = pd.read_csv(
    'Speed Dating Data.csv', encoding='cp1251')

In [13]:
df = df.drop(['id', 'idg', 'condtn', 'round', 'position', 'positin1',
                 'order', 'partner', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin',
                  'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
                  'dec_o', 'attr_o',
                  'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o',
                  'prob_o', 'met_o', 'field', 'undergra',
                  'imprelig', 'imprace',
                  'from', 'zipcode', 'career', 'sports',
                  'tvsports', 'exercise',
                  'dining', 'museums',
                  'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv',
                  'theater', 'movies', 'concerts', 'music', 'shopping',
                  'yoga', 'expnum', 'wave'], axis=1)

In [14]:
cols = ['mn_sat', 'tuition', 'income']
df = df.apply(lambda x: x.str.replace(',', '').astype(np.float) if x.name in cols else x)
df = df.apply(lambda x: x.fillna(x.mean()) if x.name in cols else x)
cols = ['field_cd', 'career_c']
df = df.apply(lambda x: x.astype(np.float) if x.name in cols else x)
df = df.apply(lambda x: x.fillna(x.mean()) if x.name in cols else x)

In [7]:
df = df.apply(lambda x: x.astype(np.float))
df = df.apply(lambda x: x.fillna(x.mean()))

In [8]:
y = df.match.astype(np.int)
X = df.drop('match', axis=1)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.4)

## Проверка скорости работы на Speed Dating Data 

In [29]:
clf = DecisionTreeClassifier(min_samples_split=2)
my_clf = MyDecisionTreeClassifier(min_samples_split=2)

In [30]:
%time clf.fit(X_train, y_train)

Wall time: 336 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [31]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')
%time my_clf.fit(X_train, y_train)

Wall time: 230 ms


## Проверка качества работы на Speed Dating Data

In [32]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.6958845383649375

In [35]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.45566742448847025

## Задание 3

In [341]:
feature_names = X.columns
pd.Series(data=clf.feature_importances_, index=feature_names).sort_values(ascending=False).head(10)

dec         0.267885
int_corr    0.081167
pid         0.057023
prob        0.050752
shar        0.037638
them_cal    0.035456
attr        0.031272
like        0.026917
sinc        0.026096
intel       0.024870
dtype: float64

In [342]:
pd.Series(data=clf.feature_importances_, index=feature_names).sort_values(ascending=False).head(10)

dec         0.267885
int_corr    0.081167
pid         0.057023
prob        0.050752
shar        0.037638
them_cal    0.035456
attr        0.031272
like        0.026917
sinc        0.026096
intel       0.024870
dtype: float64

## Задание 4

In [ ]:
params = {'max_depth': range(1,12),
          'criterion': ['gini', 'entropy'],
          'max_features': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
         }
clf = RandomForestClassifier(n_estimators=100)

random_search = GridSearchCV(clf, param_grid=params, cv=4, verbose=2)
random_search.fit(X.values, y.values)
print(random_search.best_params_)

In [11]:
print(random_search.best_params_)

{'criterion': 'gini', 'max_depth': 6, 'max_features': 0.6}


## Задание 5